In [1]:
from pathlib import Path
import pandas as pd
from pandas import DataFrame
import numpy as np

In [2]:
def get_item_info_df(src_folder: Path) -> DataFrame:
    items_df = pd.read_csv(src_folder/"items.csv")
    item_categories_df = pd.read_csv(src_folder/"item_categories.csv")
    items_info_df = items_df.merge(item_categories_df, on="item_category_id")
    return items_info_df

In [3]:
def aggregate_months(df: DataFrame, item_info_df: DataFrame) -> DataFrame:
    return (df.query("item_cnt_day > 0")
        .assign(
            date_month=lambda df: df.date.dt.month,
            date_year=lambda df: df.date.dt.year,
        )
        .assign(
            date_block_num=lambda df: (df.date_year - 2013) * 12 + df.date_month,
        )
        .assign(date_block_num=lambda df: df.date_block_num.astype(int))
        .groupby(["shop_id", "item_id", "date_block_num"])
        .agg({
            "item_cnt_day": sum, 
            "date_year": lambda r: r.iloc[0], 
            "date_month": lambda r: r.iloc[0], 
            "item_price": np.mean
        })
        .rename(columns={"item_cnt_day": "item_cnt_month"})
        .reset_index()
        .sort_values(["date_block_num", "shop_id", "item_id"])
        .assign(item_cnt_month=lambda df: df.item_cnt_month.clip(0, 20))
        .merge(item_info_df, on="item_id")
    )

In [4]:
src_folder = Path("../data/src/")

In [5]:
train = pd.read_csv("../data/train/sales.csv").assign(date=lambda df: pd.to_datetime(df.date))
val = pd.read_csv("../data/val/sales.csv").assign(date=lambda df: pd.to_datetime(df.date))

FileNotFoundError: [Errno 2] No such file or directory: '../data/train/sales.csv'

In [ ]:
item_info_df = get_item_info_df(src_folder)

In [ ]:
train_df = aggregate_months(train, item_info_df)

In [ ]:
train_df

In [ ]:
val_df = aggregate_months(val, item_info_df)

In [ ]:
val_df